In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

## Options

In [2]:
# parse options
problem = 'twelve_pieces_process.json' # 'pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = 'results'

recompute_action_states = False
load_external_movements = False

In [3]:
# generic planning options
id_only = None # 'A2_M1' # None | 'A273_M0'

low_res = False
plan_impacted = False

#     'nonlinear',
#     'linear',
#     'id_only', # 'Compute only for movement with a specific tag, e.g. `A54_M0`.'
#     'free_motion_only', # 'Only compute free motions.'
#     'propagate_only', # 'Only do state propagation and impacted movement planning.'
solve_mode = 'linear'

viz_upon_found = False

In [4]:
# client options
viewer = True
verbose = True

debug = False
diagnosis = False

watch = True
step_sim = False

disable_env = False
reinit_tool = False

write = False
save_now = False

In [5]:
from collections import namedtuple
PlanningArguments = namedtuple('PlanningArguments', ['problem', 'viewer', 'debug', 'diagnosis', 'id_only', 'solve_mode', 'viz_upon_found', 
                                             'save_now', 'write', 'plan_impacted', 'watch', 'step_sim', 'verbose'])

args = PlanningArguments(problem, viewer, debug, diagnosis, id_only, solve_mode, viz_upon_found, save_now, write, plan_impacted, watch, step_sim, verbose)

## Parse process from json

In [6]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, get_process_path, save_process

In [7]:
process = parse_process(problem, subdir=problem_subdir)
result_path = get_process_path(problem, subdir='results')
if len(process.movements) == 0:
    cprint('No movements found in process, trigger recompute actions.', 'red')
    recompute_action_states = True
if recompute_action_states:
    cprint('Recomputing Actions and States', 'cyan')
    recompute_action_states(process)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results\twelve_pieces_process.json


In [8]:
from copy import deepcopy
unsolved_process = deepcopy(process)

In [10]:
# # force load external if only planning for the free motions
# load_external_movements = load_external_movements or free_motion_only or id_only is not None
# if load_external_movements:
#     ext_movement_path = os.path.dirname(result_path)
#     cprint('Loading external movements from {}'.format(ext_movement_path), 'cyan')
#     process.load_external_movements(ext_movement_path)
#     if recompute_action_states:
#         save_process(process, result_path)
#         cprint('Recomputed process saved to %s' % result_path, 'green')

In [11]:
# from collections import defaultdict

runtime_data = {}
with open('figs/{}_runtime_data.json'.format(beam_id), 'r') as f:
    runtime_data = json.load(f)
    
# runtime_data2 = {}
# with open('figs/b4_runtime_data_linear_backward.json', 'r') as f:
#     runtime_data2 = json.load(f)
# runtime_data.update(runtime_data2)

FileNotFoundError: [Errno 2] No such file or directory: 'figs/b4_runtime_data.json'

In [12]:
runtime_data.keys()

dict_keys([])

## Start client

In [9]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
# viewer or diagnosis or view_states or watch or step_sim,
client, robot, _ = load_RFL_world(viewer=False, verbose=False)
set_initial_state(client, robot, process, disable_env=disable_env, reinit_tool=reinit_tool)

In [17]:
client.disconnect()

In [10]:
from integral_timber_joints.planning.robot_setup import GANTRY_ARM_GROUP, GANTRY_GROUP, BARE_ARM_GROUP
from compas.robots import Joint
import numpy as np

joint_names = robot.get_configurable_joint_names(group=GANTRY_ARM_GROUP)
joint_types = robot.get_joint_types_by_names(joint_names)
# 0.1 rad = 5.7 deg
joint_jump_threshold = {jt_name : np.pi/6 \
        if jt_type in [Joint.REVOLUTE, Joint.CONTINUOUS] else 0.1 \
        for jt_name, jt_type in zip(joint_names, joint_types)}

options = {
    'distance_threshold' : 0.0012,
    'frame_jump_tolerance' : 0.0012,
    'verbose' : verbose,
    'jump_threshold' : joint_jump_threshold,
#     'max_distance' : args.max_distance,
}

# Plan movements for a beam

In [15]:
from copy import copy, deepcopy
import time

# start_time = time.time()
# process = copy(unsolved_process)
# print('Copy time: {:.5f}'.format(time.time() - start_time))

start_time = time.time()
process = deepcopy(unsolved_process)
print('Deepcopy time: {:.5f}'.format(time.time() - start_time))

Deepcopy time: 3.91107


In [16]:
beam_id = 'b4'
process.get_movement_summary_by_beam_id(beam_id)

=====
Summary:
---
(0) RoboticFreeMovement(#A32_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to get clamp., traj 0) 
priority 0 | has start conf False, TCP True | has end conf False, TCP True | has traj None
---
(1) RoboticLinearMovement(#A32_M1, Linear Advance to Storage Frame of CL3 ('c1'), to get tool., traj 0) 
priority 0 | has start conf False, TCP True | has end conf True, TCP True | has traj None
---
(2) RoboticDigitalOutput(#A32_M2, Toolchanger Lock CL3 ('c1')) 
priority -1 | has start conf True, TCP True | has end conf True, TCP True
---
(3) RoboticDigitalOutput(#A32_M3, CL3 ('c1') Open Gripper to release itself from storage pad.) 
priority -1 | has start conf True, TCP True | has end conf True, TCP True
---
(4) RoboticLinearMovement(#A32_M4, Linear Retract 1 of 2 after getting CL3 ('c1') from storage., traj 0) 
priority 0 | has start conf True, TCP True | has end conf False, TCP True | has traj None
---
(5) RoboticLinearMovement(#A32_M5, Linear Retract 2 of 2 aft

In [62]:
client, robot, _ = load_RFL_world(viewer=True, verbose=False)
set_initial_state(client, robot, process, disable_env=disable_env, reinit_tool=reinit_tool)

In [25]:
pp.remove_all_debug()
client.set_robot_configuration(robot, process.robot_initial_config)

In [18]:
del options['movement_id_filter']
print(options)

{'distance_threshold': 0.0012, 'frame_jump_tolerance': 0.0012, 'verbose': True, 'jump_threshold': {'bridge1_joint_EA_X': 0.1, 'robot11_joint_EA_Y': 0.1, 'robot11_joint_EA_Z': 0.1, 'robot11_joint_1': 0.5235987755982988, 'robot11_joint_2': 0.5235987755982988, 'robot11_joint_3': 0.5235987755982988, 'robot11_joint_4': 0.5235987755982988, 'robot11_joint_5': 0.5235987755982988, 'robot11_joint_6': 0.5235987755982988}, 'debug': False, 'diagnosis': False, 'low_res': False, 'movement_planning_reattempts': 1, 'solve_timeout': 1800, 'samplig_order_counter': 0, 'lockrenderer': <pybullet_planning.interfaces.env_manager.simulation.LockRenderer object at 0x0000023F1A747C88>}


In [16]:
from integral_timber_joints.planning.run import compute_movements_for_beam_id

debug = False
diagnosis = False
verbose = True

beam_id = 'b4'
options.update({
    'debug' : debug,
    'verbose' : verbose,
    'diagnosis' : diagnosis,
    'low_res' : False,
    'movement_planning_reattempts' : 1,
    'solve_timeout' : 1800,
})

# A38M0
# PlanningArguments = namedtuple('PlanningArguments', ['problem', 'viewer', debug', 'diagnosis', 'id_only', 'solve_mode', 'viz_upon_found', 
#                                              'save_now', 'write', 'plan_impacted', 'watch', 'step_sim', 'verbose'])
args = PlanningArguments(problem, True, debug, diagnosis, 'A38_M0', 'nonlinear', viz_upon_found, save_now, write, 
                         plan_impacted, True, False, verbose)

success = compute_movements_for_beam_id(client, robot, process, beam_id, args, options=options)




* compute ['RoboticLinearMovement', 'RoboticClampSyncLinearMovement'] (priority 1, status [<MovementStatus.neither_done: 5>, <MovementStatus.one_sided: 3>])



* compute ['RoboticLinearMovement'] (priority 0, status [<MovementStatus.one_sided: 3>])
----------
(1)
Movement planning attempt 0
RoboticLinearMovement(#A32_M1, Linear Advance to Storage Frame of CL3 ('c1'), to get tool., traj 0)
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
Conf disagreement found, FK center point diff 0.00000(m), but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.
~~~~~
	Propagate states for (1) : RoboticLinearMovement(#A32_M1, Linear Advance to Storage Frame of CL3 ('c1'), to get tool., traj 1)
	$ Impacted (backward): (0) RoboticFreeMovement(#A32_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to get clamp., traj 0)
	- Altered (forward): (2) RoboticDigitalOutput(#A32_M2, Toolchanger

In [11]:
import json
from collections import defaultdict
from integral_timber_joints.planning.run import compute_movements_for_beam_id, plan_for_beam_id_with_restart
# beam_id = process.get_beam_id_from_movement_id(args.id_only)

debug = False
diagnosis = False
verbose = True

beam_id = 'b4'
options.update({
    'debug' : debug,
    'verbose' : verbose,
    'diagnosis' : diagnosis,
    'low_res' : False,
    'movement_planning_reattempts' : 1,
    'solve_timeout' : 1800,
})

# PlanningArguments = namedtuple('PlanningArguments', ['problem', 'viewer', debug', 'diagnosis', 'id_only', 'solve_mode', 'viz_upon_found', 
#                                              'save_now', 'write', 'plan_impacted', 'watch', 'step_sim', 'verbose'])

num_trails = 10
runtime_data = {}

#'linear'
# for solve_mode_ in ['nonlinear', 'linear']:
solve_mode_ = 'nonlinear'

args = PlanningArguments(problem, False, debug, diagnosis, None, solve_mode_.split('_')[0], viz_upon_found, save_now, write, 
                         plan_impacted, False, False, verbose)
all_movements = process.get_movements_by_beam_id(beam_id)

if solve_mode_ == 'linear_backward':
    options['movement_id_range'] = list(reversed(range(0, len(all_movements))))
elif solve_mode_ == 'linear_forward':
    options['movement_id_range'] = list(range(0, len(all_movements)))
    
runtime_data[solve_mode_] = {}
for attempt_i in range(num_trails):
    process = deepcopy(unsolved_process)
    success, trial_data = plan_for_beam_id_with_restart(client, robot, process, beam_id, args, options=options)
    runtime_data[solve_mode_][attempt_i] = trial_data
    with open('figs/{}_runtime_data.json'.format(beam_id), 'w') as f:
        json.dump(runtime_data, f)

print('Done')
client.disconnect()

NameError: name 'runtime_data' is not defined

In [55]:
with open('figs/{}_runtime_data.json'.format(beam_id), 'w') as f:
    json.dump(runtime_data, f)

In [54]:
for solve_mode_ in ['nonlinear', 'linear_forward', 'linear_backward']:
    print('='*20)
    for i, tdata in runtime_data[solve_mode_].items():
        print('#{}-T#{}:'.format(solve_mode_, i))
        sc = any([d['success'] for di, d in tdata.items()])
        
        total_runtime = []
        for i, trial_data in tdata.items():
            trial_profiles = trial_data['profiles']
            runtime_per_move = [sum(trial_profiles[mid]['plan_time']) for mid in trial_profiles]
            total_runtime.append(sum(runtime_per_move))
      
        cprint('{} - BT {} | time {:.2f}'.format(sc, len(tdata), sum(total_runtime)), 'green' if sc else 'red')
        print('---')

#nonlinear-T#0:
False - BT 5 | time 1811.02
---
#nonlinear-T#1:
False - BT 5 | time 2231.90
---
#nonlinear-T#2:
False - BT 4 | time 2295.43
---
#nonlinear-T#3:
False - BT 9 | time 2292.07
---
#nonlinear-T#4:
False - BT 4 | time 2332.44
---
#nonlinear-T#5:
False - BT 5 | time 1824.16
---
#nonlinear-T#6:
False - BT 9 | time 2255.36
---
#nonlinear-T#7:
True - BT 1 | time 561.54
---
#nonlinear-T#8:
False - BT 11 | time 2290.56
---
#nonlinear-T#9:
True - BT 3 | time 1081.51
---
#linear_forward-T#0:
False - BT 38 | time 1697.92
---
#linear_forward-T#1:
False - BT 42 | time 1629.65
---
#linear_forward-T#2:
True - BT 10 | time 475.35
---
#linear_forward-T#3:
False - BT 36 | time 1645.06
---
#linear_forward-T#4:
True - BT 7 | time 418.36
---
#linear_forward-T#5:
True - BT 36 | time 1414.15
---
#linear_forward-T#6:
True - BT 6 | time 206.76
---
#linear_forward-T#7:
True - BT 9 | time 381.17
---
#linear_forward-T#8:
True - BT 19 | time 771.37
---
#linear_forward-T#9:
True - BT 3 | time 123.53
---

# Diagram

In [77]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from integral_timber_joints.process import RoboticFreeMovement, RoboticLinearMovement, RoboticClampSyncLinearMovement

solve_mode_ = 'nonlinear' # linear_backward | linear_forward | nonlinear

# total_rows = 0
# for i, d in runtime_data[solve_mode_].items():
#     total_rows += len(d)+1
max_inner_loop_displayed = 6

for attempt_i, s_rdata in runtime_data[solve_mode_].items():
    if len(s_rdata) > max_inner_loop_displayed:
        num_rows = max_inner_loop_displayed+1
        half = int(max_inner_loop_displayed/2)
        selected_inners = list(range(0,half)) + list(range(len(s_rdata)-half,len(s_rdata)))
    else:
        num_rows = len(s_rdata)+1
        selected_inners = list(range(len(s_rdata)))
    
    fig = make_subplots(rows=num_rows, cols=2)
    success = any([d['success'] for di, d in s_rdata.items()])
    total_runtime = []
    failed_m_id = []
    for i in s_rdata.keys():
        trial_data = s_rdata[i]
        trial_profiles = trial_data['profiles']
        mid_keys = sorted(trial_profiles.keys(), key=int)
        runtime_per_move = [sum(trial_profiles[mid]['plan_time']) for mid in mid_keys]
        total_runtime.append(sum(runtime_per_move))
        
        for mid in mid_keys:
            if not any(trial_profiles[mid]['plan_success']):
                movement = process.get_movement_by_movement_id(trial_profiles[mid]['movement_id'][0])
                m_color = '#ff1b6b' if isinstance(movement, RoboticFreeMovement) else '#45caff'
                failed_m_id.append((mid, movement.short_summary, m_color))
                break
        else:
            failed_m_id.append((-1, 'success!', '#00ff87'))
        
        if i in selected_inners or int(i) in selected_inners:
            success_colors = ['#99C24D' if any(trial_profiles[mid]['plan_success']) else '#F18F01' for mid in mid_keys]
            row_id = selected_inners.index(int(i))+1
            fig.append_trace(go.Scatter(x=mid_keys,
                                    y=runtime_per_move,
                                    mode='markers',
                                    marker_color=success_colors,
                                    text=[process.get_movement_by_movement_id(trial_profiles[mid]['movement_id'][0]).short_summary \
                                          for mid in mid_keys], # hover text goes here
                                    name='#{}-feasibility'.format(i),
                                    ),
                          row=row_id, col=1
                        )

            fig.append_trace(go.Scatter(x=mid_keys,
                                            y=runtime_per_move,
                                            mode='markers',
                                                marker=dict(
                                                size=5,
                                                color=[trial_profiles[mid]['sample_order'][0] for mid in mid_keys], #set color equal to a variable
                                                colorscale='Viridis', # one of plotly colorscales
                                                showscale=True
                                            ),
                                            text=['S#{}-{}'.format(trial_profiles[mid]['sample_order'][0], process.get_movement_by_movement_id(trial_profiles[mid]['movement_id'][0]).short_summary) \
                                                  for mid in mid_keys], # hover text goes here
                                            name='#{}-sample order'.format(i),),
                          row=row_id, col=2
                            )
            if row_id == 1:
                fig.update_xaxes(title_text="m_id",row=row_id, col=1)
                fig.update_yaxes(title_text="runtime(s)",row=row_id, col=1)

    fig.append_trace(go.Scatter(x=list(range(len(s_rdata))),y=total_runtime), 
                     row=num_rows, col=1)
    fig.update_xaxes(title_text="trials",row=num_rows, col=1)
    fig.update_yaxes(title_text="runtime(s)",row=num_rows, col=1)
    
    fig.append_trace(go.Scatter(x=list(range(len(failed_m_id))),y=[tt[0] for tt in failed_m_id],
                                mode='markers',
                                marker_color=[tt[2] for tt in failed_m_id],
                                text=[tt[1] for tt in failed_m_id],
                               ), row=num_rows, col=2)
    fig.update_xaxes(title_text="trials",row=num_rows, col=2)
    fig.update_yaxes(title_text="failed_movement_id",row=num_rows, col=2)

    
    title = "figs/{}-{}-trail_{}_success-{}_BT-{}_time-{:.1f}".format(beam_id, solve_mode_, 
        attempt_i, success, len(s_rdata), sum(total_runtime))
    fig.update_layout(title=title)
    fig.write_html(title + ".html")
# fig.show()

In [41]:
len(failed_m_id)

207

# Save runtime data

In [106]:
runtime_data.keys()

dict_keys(['linear', 'nonlinear'])

In [18]:
from integral_timber_joints.planning.parsing import save_process_and_movements

beam_all_movements = process.get_movements_by_beam_id(beam_id)
if 'movement_id_range' not in options:
    altered_movements = beam_all_movements
else:
    altered_movements = [beam_all_movements[mid] for mid in options['movement_id_range']]
save_process_and_movements(problem, process, altered_movements, save_temp=True)

KeyError: 'movement_id_range'

# Disconnect client

In [48]:
client.disconnect()

# Plan only one movement

In [29]:
# if id_only:
#     beam_id = process.get_beam_id_from_movement_id(id_only)
#     process.get_movement_summary_by_beam_id(beam_id)

In [39]:
from integral_timber_joints.planning.stream import compute_free_movement, compute_linear_movement
from integral_timber_joints.planning.solve import compute_movement

chosen_m = process.get_movement_by_movement_id(id_only)
compute_movement(client, robot, process, chosen_m, options=lm_options, diagnosis=diagnosis)

RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)
end conf FK inconsistent (0.00005 m) with given current frame in end state.
Both start/end confs are pre-specified, problem might be too stiff to be solved.
One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.


True

In [40]:
from integral_timber_joints.planning.visualization import visualize_movement_trajectory

with pp.WorldSaver():
    visualize_movement_trajectory(client, robot, process, chosen_m, step_sim=True)

===
Viz:
RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)


Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
End state. 
